In [4]:
!pip install tensorflow_text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 7.9 MB/s eta 0:00:00a 0:00:01m


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [8]:
import pandas as pd

df = pd.read_csv("/Users/devadeva/Desktop/MNIT/Machine Learning/CodeBasics/deep-learning-keras-tf-tutorial-master/47_BERT_text_classification/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [12]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [14]:
747/4825

0.15481865284974095

In [16]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [18]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [20]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [22]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 2)

In [24]:
df_balanced['Category'].value_counts()

Category
ham     747
spam    747
Name: count, dtype: int64

In [26]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
4020,spam,U've been selected to stay in 1 of 250 top Bri...,1
721,ham,Oh is it? Send me the address,0
4221,ham,U free on sat rite? U wan 2 watch infernal aff...,0
295,ham,I accidentally deleted the message. Resend ple...,0
326,ham,No calls..messages..missed calls,0


In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [30]:
X_train.head(4)

4530      Hiya, probably coming home * weekend after next
3399      It only does simple arithmetic not percentages.
2663    Hello darling how are you today? I would love ...
2820    Don't forget who owns you and who's private pr...
Name: Message, dtype: object

In [32]:
#BERT

In [54]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name="preprocessing")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True, name="BERT_encoder")

In [56]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435168 , -0.5132726 , -0.888457  , ..., -0.7474886 ,
        -0.75314724,  0.91964495],
       [-0.87208366, -0.5054398 , -0.94446677, ..., -0.858475  ,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [58]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [62]:
cosine_similarity([e[0]],[e[3]])

array([[0.84703827]], dtype=float32)

In [64]:
cosine_similarity([e[3]],[e[4]])

array([[0.98720354]], dtype=float32)

In [66]:
#Building the Model

In [76]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Required to make BERT preprocessing work

# Load BERT preprocessing and encoder from TF Hub
bert_preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
bert_encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess_layer = hub.KerasLayer(bert_preprocess_url, name="preprocessing", trainable=False)
bert_encoder_layer = hub.KerasLayer(bert_encoder_url, name="BERT_encoder", trainable=True)

# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

# Lambda layer to wrap preprocessing
preprocessed_text = tf.keras.layers.Lambda(lambda x: bert_preprocess_layer(x))(text_input)

# BERT encoder
bert_output = bert_encoder_layer(preprocessed_text)

# Classifier head
dropout = tf.keras.layers.Dropout(0.1)(bert_output['pooled_output'])
output = tf.keras.layers.Dense(1, activation='sigmoid', name="classifier")(dropout)

# Final model
model = tf.keras.Model(inputs=text_input, outputs=output)

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()



ValueError: Exception encountered when calling layer 'preprocessing' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'preprocessing' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, ragged=False, name=text>
  • training=None